In [1]:
from pathlib import Path
import numpy as np
from pprint import pprint

from skellymodels.experimental.model_redo.managers.human import Human, HumanAspectNames
from skellymodels.experimental.model_redo.fmc_anatomical_pipeline.calculate_center_of_mass import calculate_center_of_mass_from_trajectory
from skellymodels.experimental.model_redo.fmc_anatomical_pipeline.enforce_rigid_bones import enforce_rigid_bones_from_trajectory

Input the path to raw 3D data file (as would come out of SkellyForge) 

In [2]:
path_to_data = Path(r"/Users/philipqueen/freemocap_data/recording_sessions/freemocap_sample_data/output_data/raw_data/mediapipe_3dData_numFrames_numTrackedPoints_spatialXYZ.npy")
data = np.load(path_to_data)


This is the new model info that's loaded from the YAML. Take a look  the model info at some point. It's very much in prototype.

In [3]:
from skellymodels.experimental.model_redo.tracker_info.model_info import MediapipeModelInfo
model_info = MediapipeModelInfo()

This is all you need to create a Human model. There's no data loaded in the model itself yet, but it has all the proper structures (tracked points/virtual points/all the anatomical definitions) that we currently have in the model data loaded for each aspect. 

In [4]:
human = Human(
            name="human_one", 
            model_info=model_info
            )
pprint([human.aspects])


[{'body': Aspect: body
  Anatomical Structure:
  33 tracked points
  4 virtual markers
  22 segments
  14 center of mass definitions
  16 joint hierarchies
  Trajectories: No trajectories
  Error: No reprojection error
  Metadata: : {'tracker_type': 'mediapipe'}

,
  'face': Aspect: face
  Anatomical Structure:
  478 tracked points
  No virtual markers
  No segment connections
  No center of mass definitions
  No joint hierarchy
  Trajectories: No trajectories
  Error: No reprojection error
  Metadata: : {'tracker_type': 'mediapipe'}

,
  'left_hand': Aspect: left_hand
  Anatomical Structure:
  21 tracked points
  No virtual markers
  No segment connections
  No center of mass definitions
  No joint hierarchy
  Trajectories: No trajectories
  Error: No reprojection error
  Metadata: : {'tracker_type': 'mediapipe'}

,
  'right_hand': Aspect: right_hand
  Anatomical Structure:
  21 tracked points
  No virtual markers
  No segment connections
  No center of mass definitions
  No joint hie

For the Human class, the aspects are already set (body/left hand/right hand/face) and you can access them as properties of the class

In [5]:
#For example if you want to access the marker names for an aspect, you could do what's shown below (We could make this easier to access down the line, if we feel the need)
print(f"Body marker names: {human.body.anatomical_structure.marker_names}")
print(f"Left hand marker names: {human.left_hand.anatomical_structure.marker_names}")


Body marker names: ['nose', 'left_eye_inner', 'left_eye', 'left_eye_outer', 'right_eye_inner', 'right_eye', 'right_eye_outer', 'left_ear', 'right_ear', 'mouth_left', 'mouth_right', 'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow', 'left_wrist', 'right_wrist', 'left_pinky', 'right_pinky', 'left_index', 'right_index', 'left_thumb', 'right_thumb', 'left_hip', 'right_hip', 'left_knee', 'right_knee', 'left_ankle', 'right_ankle', 'left_heel', 'right_heel', 'left_foot_index', 'right_foot_index', 'head_center', 'neck_center', 'trunk_center', 'hips_center']
Left hand marker names: ['wrist', 'thumb_cmc', 'thumb_mcp', 'thumb_ip', 'thumb_tip', 'index_finger_mcp', 'index_finger_pip', 'index_finger_dip', 'index_finger_tip', 'middle_finger_mcp', 'middle_finger_pip', 'middle_finger_dip', 'middle_finger_tip', 'ring_finger_mcp', 'ring_finger_pip', 'ring_finger_dip', 'ring_finger_tip', 'pinky_mcp', 'pinky_pip', 'pinky_dip', 'pinky_tip']


To add data from a numpy, use the `add_tracked_points_numpy` function, which splits up the data according to the information defined in the model info. There is also a `from_tracked_points_numpy` class method that creates the Human class and adds data from numpy in 1 function.
Note that now in the printed aspect below, you can see a new trajectory `3d_xyz` added. The reason I called it `3d_xyz` is because that's the name that gets tagged to that Trajectory when saving out the file later.

In [6]:
human.add_tracked_points_numpy(tracked_points_numpy_array=data)
pprint([human.body])

Calculating virtual markers: ['head_center', 'neck_center', 'trunk_center', 'hips_center']
[Aspect: body
  Anatomical Structure:
  33 tracked points
  4 virtual markers
  22 segments
  14 center of mass definitions
  16 joint hierarchies
  Trajectories: 1 trajectories: ['3d_xyz']
  Error: No reprojection error
  Metadata: : {'tracker_type': 'mediapipe'}

]


Here's how you can currently access the data. I think it's a little unwieldy and could be easier - but that's something to look at later. Below that you can also see how to get your data both as a numpy array and a pandas dataframe. 

In [7]:
print(human.body.trajectories['3d_xyz'].data)

{'nose': array([[-681.29359381, -802.25034903, 1262.99269772],
       [-651.34027664, -810.49435738, 1267.27762951],
       [-641.47428189, -801.564347  , 1261.21595382],
       ...,
       [          nan,           nan,           nan],
       [          nan,           nan,           nan],
       [          nan,           nan,           nan]]), 'left_eye_inner': array([[-650.57150898, -827.68895141, 1242.46417264],
       [-635.33332692, -829.15233053, 1240.30635672],
       [-624.42400866, -822.14763538, 1235.58582498],
       ...,
       [          nan,           nan,           nan],
       [          nan,           nan,           nan],
       [          nan,           nan,           nan]]), 'left_eye': array([[-634.75522273, -821.34347444, 1241.89878405],
       [-618.14262528, -822.96492817, 1239.94047606],
       [-608.0280186 , -816.44886629, 1235.21803821],
       ...,
       [          nan,           nan,           nan],
       [          nan,           nan,           nan],
   

In [8]:
print(f"Data as numpy: {human.body.trajectories['3d_xyz'].as_numpy}")

Data as numpy: [[[-681.29359381 -802.25034903 1262.99269772]
  [-650.57150898 -827.68895141 1242.46417264]
  [-634.75522273 -821.34347444 1241.89878405]
  ...
  [-625.25500856 -665.43615292 1501.90492245]
  [-649.9823645  -444.24042896 1674.69354616]
  [-674.70972043 -223.044705   1847.48216987]]

 [[-651.34027664 -810.49435738 1267.27762951]
  [-635.33332692 -829.15233053 1240.30635672]
  [-618.14262528 -822.96492817 1239.94047606]
  ...
  [-599.61396808 -664.82722784 1504.21446922]
  [-631.80393993 -432.14580374 1677.76303336]
  [-663.99391178 -199.46437964 1851.3115975 ]]

 [[-641.47428189 -801.564347   1261.21595382]
  [-624.42400866 -822.14763538 1235.58582498]
  [-608.0280186  -816.44886629 1235.21803821]
  ...
  [-562.37610426 -664.51492714 1514.39009845]
  [-602.44737183 -424.55647345 1685.27855438]
  [-642.5186394  -184.59801975 1856.16701031]]

 ...

 [[          nan           nan           nan]
  [          nan           nan           nan]
  [          nan           nan     

In [9]:
print(f"Data as dataframe:\n {human.body.trajectories['3d_xyz'].as_dataframe}")

Data as dataframe:
        frame          keypoint           x           y            z
0          0              nose -681.293594 -802.250349  1262.992698
1          0    left_eye_inner -650.571509 -827.688951  1242.464173
2          0          left_eye -634.755223 -821.343474  1241.898784
3          0    left_eye_outer -619.470784 -813.919184  1240.230723
4          0   right_eye_inner -680.327949 -843.743989  1251.791717
...      ...               ...         ...         ...          ...
40991   1107  right_foot_index         NaN         NaN          NaN
40992   1107       head_center         NaN         NaN          NaN
40993   1107       neck_center         NaN         NaN          NaN
40994   1107      trunk_center         NaN         NaN          NaN
40995   1107       hips_center         NaN         NaN          NaN

[40996 rows x 5 columns]


There's also some basic methods that I wrote into the Actor class that you can use to get marker or frame data. Those are demonstrated below.

While I think the way it's written is necessary for the Actor class, where we don't have a strong sense of what might be added to it, I don't think this method is ideal for the Human class. Too many strings involved. So it would probably be worth giving the human class additional methods to do the below functions.

In [10]:
print(human.get_marker_data(aspect_name=HumanAspectNames.BODY.value, type = '3d_xyz', marker_name= 'left_heel'))
print(human.get_frame(aspect_name=HumanAspectNames.BODY.value, type = '3d_xyz', frame_number=100))

[[-592.28396427  443.53927878 2299.38647366]
 [-518.93802069  485.52900465 2383.60556712]
 [-394.58993218  529.33598633 2441.02129012]
 ...
 [          nan           nan           nan]
 [          nan           nan           nan]
 [          nan           nan           nan]]
{'nose': array([-256.47083605, -842.9956903 , 1394.10112869]), 'left_eye_inner': array([-221.68670923, -883.23949189, 1396.91506593]), 'left_eye': array([-206.40797635, -881.66199505, 1399.96668789]), 'left_eye_outer': array([-191.5173028 , -879.52909193, 1402.64859052]), 'right_eye_inner': array([-257.21627509, -894.42614069, 1399.67383063]), 'right_eye': array([-265.68569786, -901.94115647, 1407.11283542]), 'right_eye_outer': array([-272.46439589, -911.32082589, 1417.66788914]), 'left_ear': array([-151.61073632, -884.35367207, 1456.64569557]), 'right_ear': array([-253.88173138, -934.65241806, 1491.54621416]), 'mouth_left': array([-233.59020109, -809.1353206 , 1428.11316439]), 'mouth_right': array([-274.76565235, 

To add reprojection error to the 3d data, first load it from the raw data folder

In [11]:
reprojection_error_path = Path(r"/Users/philipqueen/freemocap_data/recording_sessions/freemocap_sample_data/output_data/raw_data/mediapipe_3dData_numFrames_numTrackedPoints_reprojectionError.npy")
reprojection_error_data = np.load(reprojection_error_path)

We can add the reprojection error from a numpy array using `add_reprojection_error_numpy`, the same way we did from `tracked_points`.

In [12]:
human.add_reprojection_error_numpy(reprojection_error_data=reprojection_error_data)

Now we can get the error data by frame or marker

In [13]:
print(human.get_error_marker(aspect_name=HumanAspectNames.BODY.value, marker_name= 'left_heel'))
print(human.get_error_frame(aspect_name=HumanAspectNames.BODY.value, frame_number=100))

[25.52732232 17.65498045 13.86470385 ...         nan         nan
         nan]
{'nose': 5.746020473415346, 'left_eye_inner': 5.90244210284299, 'left_eye': 5.813339899127283, 'left_eye_outer': 5.656964136384929, 'right_eye_inner': 6.838173316247693, 'right_eye': 7.303391546297601, 'right_eye_outer': 7.579633577886159, 'left_ear': 5.395652581201134, 'right_ear': 6.981296190908798, 'mouth_left': 5.73569256588852, 'mouth_right': 6.010359255994679, 'left_shoulder': 10.209152768874075, 'right_shoulder': 7.507343032311457, 'left_elbow': 15.799318229136006, 'right_elbow': 22.819596236039697, 'left_wrist': 20.216664636915368, 'right_wrist': 85.12797579441322, 'left_pinky': 19.499122752521046, 'right_pinky': 100.0126877968987, 'left_index': 18.25631177921259, 'right_index': 104.39363726455751, 'left_thumb': 17.114486943363136, 'right_thumb': 97.36284714160382, 'left_hip': 6.980321597510373, 'right_hip': 7.2618867088386025, 'left_knee': 3.2968320942090963, 'right_knee': 12.394759685267323, 'left_

The following functions are what exist in our anatomical pipeline at the moment: calculating center of mass and enforcing rigid bones. For both, after the data is calculated, they get added as Trajectories to the relevant Aspect

In [14]:
for aspect in human.aspects.values():
    aspect.calculate_center_of_mass()

Calculating center of mass for aspect: body
Missing center of mass definitions for aspect face, skipping center of mass calculation
Missing center of mass definitions for aspect left_hand, skipping center of mass calculation
Missing center of mass definitions for aspect right_hand, skipping center of mass calculation


In [15]:
for aspect in human.aspects.values():
    aspect.enforce_rigid_bones()

Enforcing rigid bones for aspect: body
Missing segment connections for aspect face, skipping rigid bone enforcement
Missing segment connections for aspect left_hand, skipping rigid bone enforcement
Missing segment connections for aspect right_hand, skipping rigid bone enforcement


Below you can see extra trajectories added to the body after running the above functions

In [16]:
for aspect in human.aspects.values():
    pprint({aspect.name: aspect.trajectories})

{'body': {'3d_xyz': Trajectory with 1108 frames and 37 markers,
          'rigid_3d_xyz': Trajectory with 1108 frames and 37 markers,
          'segment_com': Trajectory with 1108 frames and 14 markers,
          'total_body_com': Trajectory with 1108 frames and 1 markers}}
{'face': {'3d_xyz': Trajectory with 1108 frames and 478 markers}}
{'left_hand': {'3d_xyz': Trajectory with 1108 frames and 21 markers}}
{'right_hand': {'3d_xyz': Trajectory with 1108 frames and 21 markers}}


Each saving function is a method in the Actor baseclass. They leverage the `as_numpy` and `as_dataframe` properties that each Trajectory has. We basically iterate over each Trajectory in an Aspect, and save it out, using the metadata stored within each aspect/trajectory to dynamically build the file names. The last function builds the 'big' CSV we have with all the data.

And these three functions are all we need to save out similar/almost equivalent data to what we currently save out for freemocap in the output data folder

In [17]:
human.save_out_numpy_data()
human.save_out_csv_data()
human.save_out_all_data_csv()


Saving out numpy: mediapipe body 3d_xyz
Saving out numpy: mediapipe body total_body_com
Saving out numpy: mediapipe body segment_com
Saving out numpy: mediapipe body rigid_3d_xyz
Saving out numpy: mediapipe face 3d_xyz
Saving out numpy: mediapipe left_hand 3d_xyz
Saving out numpy: mediapipe right_hand 3d_xyz
Saving out CSV: mediapipe body 3d_xyz
Saving out CSV: mediapipe body total_body_com
Saving out CSV: mediapipe body segment_com
Saving out CSV: mediapipe body rigid_3d_xyz
Saving out CSV: mediapipe face 3d_xyz
Saving out CSV: mediapipe left_hand 3d_xyz
Saving out CSV: mediapipe right_hand 3d_xyz
Data successfully saved to 'freemocap_data_by_frame.csv'
